## Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes  import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [2]:
train = pd.read_csv('C:/Users/Ziyaad/Desktop/Classification predict/train.csv')
test = pd.read_csv('C:/Users/Ziyaad/Desktop/Classification predict/test.csv')

In [3]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15819 entries, 0 to 15818
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  15819 non-null  int64 
 1   message    15819 non-null  object
 2   tweetid    15819 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 370.9+ KB


In [5]:
print(train)

       sentiment                                            message  tweetid
0              1  PolySciMajor EPA chief doesn't think carbon di...   625221
1              1  It's not like we lack evidence of anthropogeni...   126103
2              2  RT @RawStory: Researchers say we have three ye...   698562
3              1  #TodayinMaker# WIRED : 2016 was a pivotal year...   573736
4              1  RT @SoyNovioDeTodas: It's 2016, and a racist, ...   466954
...          ...                                                ...      ...
15814          1  RT @ezlusztig: They took down the material on ...    22001
15815          2  RT @washingtonpost: How climate change could b...    17856
15816          0  notiven: RT: nytimesworld :What does Trump act...   384248
15817         -1  RT @sara8smiles: Hey liberals the climate chan...   819732
15818          0  RT @Chet_Cannon: .@kurteichenwald's 'climate c...   806319

[15819 rows x 3 columns]


## Splitting out the X variable from the target

In [6]:
y = train['sentiment']
X = train['message']

## Turning text into something your model can read

In [7]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

## Splitting the training data into a training and validation set

In [8]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size= 0.25,shuffle=True, stratify=y, random_state=42)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_vectorized)
X_train_tfidf.shape

(15819, 23639)

## Training the model and evaluating using the validation set 

## Checking the performance of our model on the validation set

In [11]:
rfc = LinearSVC()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [13]:
f1_score(y_val, rfc_pred, average="macro")

0.6494662574541193

## Getting our test set ready 

In [14]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [15]:
y_pred = rfc.predict(test_vect)

In [16]:
test['sentiment'] = y_pred

In [17]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


## Creating an output csv for submission

In [18]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)